![Erudio logo](img/Erudio-logo.png)

---

![NumPy logo](img/numpylogo.svg)

# Boolean and Filtered Selection

In earlier modules we encountered views created by slicing.  But NumPy also provides a number of ways to view arrays that relate to the specific values in elements or to other calculated predicates

Similar to the selections we have looked at already, many of these techniques create view of data without copying memory.

## Predicates on Elements

Let us start with a very simple predicative selection.

In [ ]:
import numpy as np
arr = 3 * np.arange(1, 8.)
print(arr)
arr[arr > 10]

Much of the power of selecting by values is that the selection can be used for modification as well.

In [ ]:
arr[arr > 10] /= 5
arr

In [ ]:
arr[arr >= 3.5] = arr[arr < 3].mean()
arr

## Boolean Arrays

Under the hood, when we use predicates, we create *Boolean arrays* to express the truth of the predicate relative to each element.

In [ ]:
arr = 3 * np.arange(1, 8.)
print(arr > 10)
print(arr < np.median(arr))

As a consequence of predicative indexing really just being on Boolean arrays, those Booleans do not have to originate with the same elements, or same array even, as the ones they index.

In [ ]:
arr = np.random.randint(1, 10, 30).reshape(3, 10)
print(arr)

arr[0, arr[2] > arr[1]] *= 10
print("\nFirst row x10 if third row is bigger than second")
print(arr)

## `np.where`

A powerful way of expressing a choice of mutations is with the function `np.where()`.  There are two different modes of operation of the function: with one Boolean array argument and with a Boolean array and "when true" / "when false" arguments.

The one argument form returns a tuple of arrays of indices, one per dimension; these line up in correspondence to identify the elements of interest.  Because these matching elements might occur anywhere without respect to the shape of the original array, the new array becomes a 1-D array.  

NumPy does not support *ragged arrays* which will usually result if `np.where(cond)` attempted to preserve shape.

In [ ]:
arr = np.random.randint(0, 10, 45).reshape(3, 3, 5)
print(arr)

The component dimension arrays are usually useful as an intermediate step to getting at underlying values.

In [ ]:
indices = np.where(arr % 3 == 0)
indices

In [ ]:
arr[indices]

### Three-argument `np.where`

Some NumPy functions (we already saw `np.eye()` in passing) have notably different behaviors depending on the callng signature or argument types.  When `np.where()` is provided *when_true* and *when_false* arguments it returns an array with the same shape as the original array.

Often, but not always, the *when_true* and *when_false* arguments are calculations derived from the array(s) involved in the predicate.

In [ ]:
# Only use array for predicate
arr = np.random.randint(0, 10, 18).reshape(3, 6)
print(arr)
# Round every value to the min or max of the range
np.where(arr < 5, 0, 9)

In [ ]:
# Preserve array in one branch, round in other
np.where(arr < 5, arr, 9)

In [ ]:
# Utilize the array in the both branches
np.where(arr < 5, -arr, 3*arr)

In [ ]:
# Use two arrays in the comparison. Pick result from one or other
arr1 = np.random.randint(0, 10, 10)
arr2 = np.random.randint(0, 10, 10)
print(arr1)
print(arr2)
np.where(arr1 < arr2, arr2, arr1)

In [ ]:
# In the specific example, we could also spell this as:
print(np.maximum(arr1, arr2))

# np.maximum() is different from np.max()
np.max([arr1, arr2])

## Composing Predicates

For technical reasons, we cannot use the Python logical operators `and`, `or`, and `not` to combine predicates.  But we can use the equivalent bit-wise logical operators. Because of operator precedence, you need to surround each component clause with parentheses.

In [ ]:
# The numbers up to 100 that are divisible by 3 but not 2
arr = np.arange(100)
arr[(arr % 3 == 0) & ~(arr % 2 == 0)]

In [ ]:
# The numbers up to 100 that are divisible by 3 or 5 and greater than 75
arr[((arr % 3 == 0) | (arr % 5 == 0)) & (arr > 75)]

## Contructing Index Tuples

We saw that 1-argument `np.where()` returns tuples of coordinates that may be used for indexing.  We can also construct such tuples other ways, such as by hand. In fact, these tuples do not need to contain arrays specifically, any (nested) sequence will do.

In [ ]:
arr = np.arange(1, 37).reshape(6, 6)
arr

The indices need not be in ascending order. Moreover, neither the single dimension indices,
nor their combination need be unique.

In [ ]:
rows = [1, 1, 3, 4, 4, 5, 5, 4]
cols = [5, 1, 4, 1, 3, 3, 3, 4]
arr[rows, cols]

This tuple of dimensional indices used as an array index selects elements, and they can be modified as well as copied into a new array.

In [ ]:
arr[rows, cols] *= 100
arr

A really interesting feature of these paired (or multiple) aligned dimension indices is that they can have a shape other than one dimensional.

In [ ]:
arr = np.arange(1, 37).reshape(6, 6)
rows = [[1, 1, 3, 4], [4, 5, 5, 4]]
cols = [[5, 1, 4, 1], [3, 3, 3, 4]]
arr[rows, cols]

However, if used as idices for a modification, the dimensions of the index arrays does not matter, only their alignment with each other.  This makes sense if you think about it: the shape of the result is still the shape of the original array.

In [ ]:
arr[rows, cols] *= -1
arr

# Exercises

In these exercises, we with to perform selection or modification using  enhanced selection techniques.

In [ ]:
from src.numpy_exercises import *

Using the same 4-column array of quadrants in radians that we saw in previous exercises, transform the array into the sine of all elements whose cosine is non-negative, and to zero where the cosine is negative.

In [ ]:
arr = ex3_1.arr.copy()
ex3_1

In [ ]:
ex3_1.result

There are different approaches to the last problem.  One obvious approach involves mutating the array using a mask.  Another approach is to perform the modification in one expression.  Try to solve the problem in the other approach from what you did initially.

In [ ]:
arr = ex3_1.arr.copy()

---

Select all the radian values in the now-familiar quadrant array where both the sine and cosine of that value is negative.

In [ ]:
arr = ex3_2.arr.copy()
ex3_2

EXTRA CREDIT: Try to present the collection of radian values in ascending order using NumPy.

In [ ]:
arr = ex3_2.arr.copy()

---
This exercise may require several lines of code.  Replace the radian values whose sine and cosine are both negative with the smallest value in the fourth quadrant that does not have this property.

Note that we wish to do this *exactly* and also using generalizable techniques.  That is, don't just type in the number you see in the illustration (which will be off by floating point rounding if you do it that way).  But also do not simply select a specific row and column to get the value by manual selection.

In [ ]:
arr = ex3_3.arr.copy()
print(ex3_3.result)

---

The array in `ex3_4` simply contains random integers.

In [ ]:
arr = np.random.randint(0, 10, 40).astype(float).reshape(4, 10)
arr

### Sieve of Erotosthenes

One of the most famous and oldest known algorithms is the Sieve of Erotosthenes for finding prime numbers.  The general idea is to selectively "strike out" the multiples of each prime, starting from 2, then what is left over after all those passes is prime.

Wikipedia has a nice illustration, copied here.  The saturated/unsaturated for prime/composite shows the algorithm clearly.

![Sieve animation](img/Sieve_of_Eratosthenes_animation.gif)

(c) [CC-BY-SA](https://commons.wikimedia.org/wiki/File:Sieve_of_Eratosthenes_animation.gif)

In plain Python, we can implement this algorithm easily enough.

In [ ]:
def py_sieve(limit):
    # All positions start out with prime flag
    is_prime = [True]*limit
    # Zero and one are "non-prime"
    is_prime[0] = is_prime[1] = False
    # It suffices to search up to sqrt of the limit
    for d in range(2, int(limit**0.5) + 1):
        # No need to strike out multiples of known composites
        if is_prime[d]:
            # For every multiple flag as non-prime (composite)
            for n in range(d*d, limit, d):
                is_prime[n] = False  
    return is_prime

is_prime = py_sieve(10_000_000)

The algorithm works, just not particularly quickly.  You should be able to speed it up by about 50 times using NumPy.

* Try to make your implementation as fast as the provided one
* Try to make your implementation *faster* than the provided one

In [ ]:
is_prime[37], is_prime[38]

In [ ]:
%timeit py_sieve(10_000_000)

In [ ]:
assert np.all(py_sieve(10_000_000) == numpy_sieve(10_000_000))
%timeit numpy_sieve(10_000_000)

In [ ]:
# Your efficient implementation using NumPy
def numpy_sieve_ex(limit):
    ...
    return is_prime

# How fast is your implementation?
%timeit numpy_sieve_ex(10_000_000)

Using the function you have implemented above, select only the prime numbers greater than 6,000 from `ex3_4.arr`.  This array contains 100 random integers between 5,000 and 10,000.

In [ ]:
arr = ex3_4.arr.copy()
ex3_4

In [ ]:
ex3_4.result

---

Materials licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/) by the authors